# Packages

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import optuna
from sklearn.model_selection import KFold
from sklearn.base import clone
import joblib
from sklearn.metrics import root_mean_squared_error, r2_score
from typing import List, Dict, Optional
import scripts.ml_utils as mlu
import fs

# Reading data

In [5]:
IMTERIM_DIR = fs.open_fs("../data/interim")
TRAIN_CSV_DIR = IMTERIM_DIR.getsyspath("use_to_train.csv")
TEST_CSV_DIR = IMTERIM_DIR.getsyspath("use_to_test.csv")
VALIDATION_CSV_DIR = IMTERIM_DIR.getsyspath("use_to_val.csv")

In [6]:
NEW_MODELS_DIR = fs.open_fs("../models/new")

In [7]:
train = pd.read_csv(TRAIN_CSV_DIR)
test = pd.read_csv(TEST_CSV_DIR)
validation = pd.read_csv(VALIDATION_CSV_DIR)

In [ ]:
# var_int = train.select_dtypes(include=['int', 'float']).columns.tolist()
# feature_names_int = train.drop('price', axis=1).select_dtypes(include=['int', 'float']).columns.tolist()
# feature_names_cat = train.select_dtypes(include=['object']).columns.tolist()

In [8]:
feature_names_int = ['horsepower', 'displacement', 'torque', 'wheels', 'km', 'age']
feature_names_cat = ['navigation_system', 'rear_sensor', 'push_start', 'turbo', 'body_type']
# # var_int = feature_names_int


# Split

In [9]:
X_train = train.drop('price', axis = 1)
y_train = train['price']
X_val = validation.drop('price', axis = 1)
y_val = validation['price']

In [10]:
X_test = test.drop('price', axis = 1)
y_test = test['price']

# Pipeline

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.base import RegressorMixin, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OrdinalEncoder

In [12]:
# podriamos cambiar el nombre de la función porque hay una función con el mismo nombre en sklearn
def make_pipeline(
    regressor: RegressorMixin,
    feature_names_int: List[str], 
    feature_names_cat: List[str], 
    *, 
    cat_encoder: Optional[TransformerMixin] = None
) -> Pipeline:
    """
    Create a sklearn machine learning pipeline for preprocessing and regression.

    This function constructs a pipeline that preprocesses integer and categorical columns,
    then applies a regression model. Integer columns are scaled using `RobustScaler`, 
    and categorical columns are encoded using `OrdinalEncoder` or a custom encoder if provided.
    The regression step uses the provided regressor.

    Parameters
    ----------
    regressor : sklearn.base.RegressorMixin
        A scikit-learn compatible regressor that will be used as the final step of the pipeline.
    
    feature_names_int : list of str
        List of names of integer columns to be scaled.
    
    feature_names_cat : list of str
        List of names of categorical columns to be encoded.
    
    cat_encoder : sklearn.base.TransformerMixin, optional
        A transformer for encoding categorical features. If not provided, `OrdinalEncoder` 
        will be used by default.

    Returns
    -------
    sklearn.pipeline.Pipeline
        A scikit-learn `Pipeline` object that performs preprocessing and regression.
        
    Notes
    -----
    - Columns not included in `feature_names_int` or `feature_names_cat` will be dropped

    Examples
    --------
    >>> from sklearn.linear_model import LinearRegression
    >>> feature_names_int = ['age', 'salary']
    >>> feature_names_cat = ['gender', 'profession']
    >>> pipeline = make_pipeline(LinearRegression(), feature_names_int, feature_names_cat)
    >>> pipeline.fit(X_train, y_train)
    >>> predictions = pipeline.predict(X_test)
    """
    return Pipeline((
        (
            "preprocessing",
            ColumnTransformer((
                ("int", RobustScaler(), feature_names_int),
                (
                    "cat",
                    OrdinalEncoder() if cat_encoder is None else cat_encoder,
                    feature_names_cat,
                ),
            ),
            verbose_feature_names_out=False),
        ),
        ("regressor", regressor),
    )).set_output(transform="pandas")

# Model comparison (no tuning)

## Catboost (predicting expected value)

In [13]:
from catboost import CatBoostRegressor

In [14]:
cat_reg = CatBoostRegressor(
    iterations=100,
    learning_rate=0.3,
    bootstrap_type =  "MVS",
    cat_features=feature_names_cat
)

catboost_pipeline = make_pipeline(cat_reg, feature_names_int, feature_names_cat, cat_encoder='passthrough')
catboost_pipeline.fit(X_train, y_train)

0:	learn: 104653.0236578	total: 48.5ms	remaining: 4.8s
1:	learn: 92407.7801589	total: 50.9ms	remaining: 2.49s
2:	learn: 80351.5487205	total: 53.2ms	remaining: 1.72s
3:	learn: 72555.3830345	total: 55.8ms	remaining: 1.34s
4:	learn: 66673.1261827	total: 58.8ms	remaining: 1.12s
5:	learn: 61792.6520823	total: 60.7ms	remaining: 951ms
6:	learn: 59295.5256495	total: 62.4ms	remaining: 829ms
7:	learn: 56573.4889009	total: 64ms	remaining: 736ms
8:	learn: 54508.0036312	total: 65.7ms	remaining: 665ms
9:	learn: 53108.8616919	total: 67.7ms	remaining: 610ms
10:	learn: 52150.1281192	total: 69.3ms	remaining: 561ms
11:	learn: 51037.3439947	total: 70.8ms	remaining: 519ms
12:	learn: 50221.0841572	total: 72.3ms	remaining: 484ms
13:	learn: 49258.9353632	total: 74ms	remaining: 454ms
14:	learn: 48097.0497997	total: 75.7ms	remaining: 429ms
15:	learn: 47889.4188183	total: 76.8ms	remaining: 403ms
16:	learn: 47333.4314287	total: 78.5ms	remaining: 383ms
17:	learn: 46568.3405680	total: 80.2ms	remaining: 365ms
18:	le

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=(('int', RobustScaler(),
                                                  ['horsepower', 'displacement',
                                                   'torque', 'wheels', 'km',
                                                   'age']),
                                                 ('cat', 'passthrough',
                                                  ['navigation_system',
                                                   'rear_sensor', 'push_start',
                                                   'turbo', 'body_type'])),
                                   verbose_feature_names_out=False)),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x7b024ecdbdd0>)])

In [15]:
preds_val = catboost_pipeline.predict(X_val)
mlu.get_metrics_pd(y_val, preds_val, 'CatBoost Pipeline Validation')

,CatBoost Pipeline Validation
MSE,"2,667,709,358.05"
R^2,0.811
MAE,"33,029.95"
RMSE,"51,649.87"


In [16]:
# save model
CATBOOST_DIR = NEW_MODELS_DIR.getsyspath('3_catboost_bcu.joblib')
joblib.dump(catboost_pipeline, CATBOOST_DIR)

['/home/lenovo/Documents/MCD/ml1/models/new/3_catboost_bcu.joblib']

## Catboost (predicting interval)

In [17]:
quantile_levels = [0.5, 0.75]
quantile_str = str(quantile_levels).replace('[','').replace(']','')

cat_int_reg = CatBoostRegressor(
    loss_function=f'MultiQuantile:alpha={quantile_str}',
    thread_count= 4,
    cat_features= feature_names_cat,
    bootstrap_type =  "MVS",
    # iterations=26, learning_rate=0.1
    iterations=1000, learning_rate=0.3
)
cat_int_pipeline = make_pipeline(cat_int_reg, feature_names_int, feature_names_cat, cat_encoder='passthrough')
cat_int_pipeline.fit(X_train, y_train)

0:	learn: 35660.6186747	total: 3.38ms	remaining: 3.38s
1:	learn: 29936.3602451	total: 7.04ms	remaining: 3.51s
2:	learn: 25878.6662261	total: 10.4ms	remaining: 3.44s
3:	learn: 22467.3144834	total: 13.5ms	remaining: 3.35s
4:	learn: 20277.3724367	total: 17ms	remaining: 3.39s
5:	learn: 18368.5207284	total: 19.9ms	remaining: 3.3s
6:	learn: 16984.7645902	total: 22.8ms	remaining: 3.23s
7:	learn: 16279.0603452	total: 25.8ms	remaining: 3.2s
8:	learn: 15723.4955916	total: 29.3ms	remaining: 3.22s
9:	learn: 15293.0381011	total: 32ms	remaining: 3.17s
10:	learn: 14874.2109815	total: 34.7ms	remaining: 3.12s
11:	learn: 14637.9370484	total: 37.3ms	remaining: 3.07s
12:	learn: 14435.6764774	total: 40.3ms	remaining: 3.06s
13:	learn: 14297.0750204	total: 43.4ms	remaining: 3.06s
14:	learn: 13928.2231723	total: 47.3ms	remaining: 3.11s
15:	learn: 13759.7752682	total: 50.6ms	remaining: 3.11s
16:	learn: 13572.3527239	total: 53.6ms	remaining: 3.1s
17:	learn: 13304.1456487	total: 56.6ms	remaining: 3.09s
18:	learn

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=(('int', RobustScaler(),
                                                  ['horsepower', 'displacement',
                                                   'torque', 'wheels', 'km',
                                                   'age']),
                                                 ('cat', 'passthrough',
                                                  ['navigation_system',
                                                   'rear_sensor', 'push_start',
                                                   'turbo', 'body_type'])),
                                   verbose_feature_names_out=False)),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x7b024c9519d0>)])

In [18]:
inter_pred = cat_int_pipeline.predict(X_train)

predictions = y_train.to_frame(name="y_true") # the "ground truth" column
predictions["pi_median"] = inter_pred[:, 0]
predictions["pi_upper"] = inter_pred[:, 1]
predictions["avg"] = ((predictions.pi_median + predictions.pi_upper)/2)
predictions

,y_true,pi_median,pi_upper,avg
0,451999,453621.217289,465998.332332,459809.774810
1,281999,283515.207976,276730.783842,280122.995909
2,224999,224456.056119,226504.041646,225480.048882
3,171999,189828.001832,191933.105431,190880.553631
4,199999,195865.932351,199880.709152,197873.320751
...,...,...,...,...
932,319999,322194.692601,327053.522898,324624.107750
933,161999,161976.653061,162023.914230,162000.283645
934,377999,366626.371402,377811.957859,372219.164630
935,285999,273746.711700,285002.354658,279374.533179


In [19]:
preds_val = cat_int_pipeline.predict(X_val)

mlu.get_metrics_pd(y_val, preds_val[:, 0], 'Catboost Interval Pipeline Validation')

,Catboost Interval Pipeline Validation
MSE,"2,339,653,769.68"
R^2,0.834
MAE,"30,683.91"
RMSE,"48,369.97"


In [20]:
# save model
CATBOOST_INTERVAL_DIR = NEW_MODELS_DIR.getsyspath('3_catboost_interval_bcu.joblib')
joblib.dump(cat_int_pipeline, CATBOOST_INTERVAL_DIR)

['/home/lenovo/Documents/MCD/ml1/models/new/3_catboost_interval_bcu.joblib']

## LGBM

In [21]:
lgbm_reg = LGBMRegressor(
    objective='quantile',
    alpha=0.5
)

lgbm_pipeline = make_pipeline(lgbm_reg, feature_names_int, feature_names_cat)
lgbm_pipeline.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000297 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 534
[LightGBM] [Info] Number of data points in the train set: 937, number of used features: 11
[LightGBM] [Info] Start training from score 281999.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=(('int', RobustScaler(),
                                                  ['horsepower', 'displacement',
                                                   'torque', 'wheels', 'km',
                                                   'age']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['navigation_system',
                                                   'rear_sensor', 'push_start',
                                                   'turbo', 'body_type'])),
                                   verbose_feature_names_out=False)),
                ('regressor', LGBMRegressor(alpha=0.5, objective='quantile'))])

In [22]:
preds_val = lgbm_pipeline.predict(X_val)
mlu.get_metrics_pd(y_val, preds_val, 'LGBM Pipeline Validation')

,LGBM Pipeline Validation
MSE,"2,699,288,978.85"
R^2,0.809
MAE,"32,586.79"
RMSE,"51,954.68"


In [23]:
# save model
LGBM_DIR = NEW_MODELS_DIR.getsyspath('3_lgbm_bcu.joblib')
joblib.dump(lgbm_pipeline, LGBM_DIR)

['/home/lenovo/Documents/MCD/ml1/models/new/3_lgbm_bcu.joblib']

# Hyperparameters

In [24]:
from sklearn.base import clone

## Catboost

In [25]:
def catboost_objective(trial:optuna.trial.Trial) -> float:
    params = {
        "regressor__n_estimators": trial.suggest_int('regressor__n_estimators', 100, 1000, log=True),
        "regressor__learning_rate": trial.suggest_float("regressor__learning_rate", 1e-3, 0.3, log=True),
        "regressor__depth": trial.suggest_int("regressor__depth", 1, 16),
        "regressor__subsample": trial.suggest_float("regressor__subsample", 0.05, 1.0),
        "regressor__colsample_bylevel": trial.suggest_float("regressor__colsample_bylevel", 0.05, 1.0),
        "regressor__min_data_in_leaf": trial.suggest_int("regressor__min_data_in_leaf", 1, 100)
    }
    
    quantile_levels = [0.5, 0.75]
    quantile_str = str(quantile_levels).replace('[','').replace(']','')

    model = CatBoostRegressor(
        loss_function=f'MultiQuantile:alpha={quantile_str}',
        thread_count= 4,
        cat_features= feature_names_cat,
        bootstrap_type =  "MVS",
        verbose=0
    )
    pipeline = make_pipeline(model, feature_names_int, feature_names_cat, cat_encoder='passthrough')
    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    rmse = root_mean_squared_error(y_test, predictions[:, 0])

    return rmse

In [ ]:
%%time
cat_study = optuna.create_study(direction='minimize', study_name='catboost')
cat_study.optimize(catboost_objective, n_trials=30)

[I 2024-10-20 14:59:12,151] A new study created in memory with name: catboost


In [ ]:
print('Best hyperparameters:', cat_study.best_params)
print('Best RMSE:', cat_study.best_value)

Best hyperparameters: {'regressor__n_estimators': 50, 'regressor__learning_rate': 0.05907445114637935, 'regressor__depth': 13, 'regressor__subsample': 0.9852861545905143, 'regressor__colsample_bylevel': 0.5082056157389105, 'regressor__min_data_in_leaf': 21}
Best RMSE: 73921.1925406709


### verify

In [ ]:
cat_int_reg = CatBoostRegressor(
    loss_function=f'MultiQuantile:alpha={quantile_str}',
    thread_count= 4,
    cat_features= feature_names_cat,
    bootstrap_type =  "MVS"
)
cat_int_pipeline = make_pipeline(cat_int_reg, feature_names_int, feature_names_cat, cat_encoder='passthrough')
cat_int_pipeline.set_params(**cat_study.best_params)
cat_int_pipeline.fit(X_train, y_train)

0:	learn: 42456.4869746	total: 396ms	remaining: 19.4s
1:	learn: 40250.5289956	total: 799ms	remaining: 19.2s
2:	learn: 38321.3003513	total: 1.12s	remaining: 17.6s
3:	learn: 36449.2985519	total: 1.5s	remaining: 17.3s
4:	learn: 34923.6114924	total: 1.55s	remaining: 13.9s
5:	learn: 33578.9067788	total: 1.55s	remaining: 11.4s
6:	learn: 31953.5257957	total: 1.98s	remaining: 12.1s
7:	learn: 30424.0120729	total: 2.34s	remaining: 12.3s
8:	learn: 28930.4014987	total: 2.64s	remaining: 12s
9:	learn: 27600.3869345	total: 3.04s	remaining: 12.2s
10:	learn: 26555.7969806	total: 3.06s	remaining: 10.8s
11:	learn: 25344.4819276	total: 3.46s	remaining: 10.9s
12:	learn: 24209.2856591	total: 3.84s	remaining: 10.9s
13:	learn: 23013.2582268	total: 4.16s	remaining: 10.7s
14:	learn: 22012.4023830	total: 4.43s	remaining: 10.3s
15:	learn: 21072.8914898	total: 4.79s	remaining: 10.2s
16:	learn: 20301.6930065	total: 5.15s	remaining: 9.99s
17:	learn: 19394.2739413	total: 5.48s	remaining: 9.74s
18:	learn: 18656.183919

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=(('int', RobustScaler(),
                                                  ['fuel_consumption_km_l',
                                                   'horsepower', 'displacement',
                                                   'gears', 'torque',
                                                   'passengers', 'doors',
                                                   'wheels', 'km', 'age']),
                                                 ('cat', 'passthrough',
                                                  ['touch_screen',
                                                   'navigation_system',
                                                   'push_start',
                                                   'rear_cupholders', 'sunroof',
                                                   'rear_sensor', 'start_stop',
                                                   'turbo', 'seat_material',
                                                   'trunk_opening',
                                                   'headlights', 'body_type',
                                                   'electric_parking_brake',
                                                   'electric_locks'])),
                                   verbose_feature_names_out=False)),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x7f5ea02d1ae0>)])

In [ ]:
preds_val = cat_int_pipeline.predict(X_val)
mlu.get_metrics_pd(y_val, preds_val[:, 0], 'Catboost Pipeline Optuna Metrics')

,Catboost Pipeline Optuna Metrics
MSE,3.356323e+09
R^2,7.620000e-01
MAE,3.519438e+04
RMSE,5.793378e+04


In [ ]:
# save model
CATBOOST_OPTUNA_DIR = NEW_MODELS_DIR.getsyspath('3_catboost_optuna_bcu.joblib')
joblib.dump(cat_int_pipeline, CATBOOST_OPTUNA_DIR)

## LGBM

In [ ]:
def lgbm_objective(trial: optuna.trial.Trial) -> float:
    params = {
        'regressor__n_estimators': trial.suggest_int('regressor__n_estimators', 100, 1000, log=True),
        'regressor__learning_rate': trial.suggest_float('regressor__learning_rate', 1e-3, 0.5, log=True),
        #'num_leaves': trial.suggest_int('num_leaves', 8, 256, log=True),
        'regressor__max_depth': trial.suggest_int('regressor__max_depth', 5, 16, log=True),
        'regressor__colsample_bytree': trial.suggest_float("regressor__colsample_bytree", 0.1, 1),
        'regressor__reg_alpha': trial.suggest_float('regressor__reg_alpha', 1e-8, 100, log=True),
        'regressor__reg_lambda': trial.suggest_float('regressor__reg_lambda', 1e-8, 100,log=True),
        'regressor__min_split_gain': trial.suggest_float('regressor__min_split_gain', 1e-8, 100,log=True),
        'regressor__subsample': trial.suggest_float("regressor__subsample", 0.1, 1),
        'regressor__min_child_samples': trial.suggest_int('regressor__min_child_samples', 20, 1000, log=True)}

    model = LGBMRegressor(
        objective='quantile',
        alpha=0.5,
        verbose=0
    )
    pipeline = make_pipeline(model, feature_names_int, feature_names_cat)
    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    rmse = root_mean_squared_error(y_test, predictions)
    return rmse

In [ ]:
%%time
lgbm_study = optuna.create_study(direction='minimize', study_name='lgbm')
lgbm_study.optimize(lgbm_objective, n_trials=30)

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
print('Best hyperparameters:', lgbm_study.best_params)
print('Best RMSE:', lgbm_study.best_value)

Best hyperparameters: {'regressor__n_estimators': 989, 'regressor__learning_rate': 0.08295730914724425, 'regressor__max_depth': 8, 'regressor__colsample_bytree': 0.5848404052757482, 'regressor__reg_alpha': 8.442355086163822e-05, 'regressor__reg_lambda': 0.21318302871006012, 'regressor__min_split_gain': 0.00012883060884643252, 'regressor__subsample': 0.1949654146226087, 'regressor__min_child_samples': 20}
Best RMSE: 48353.25741059976


### verify

In [ ]:
lgbm_pipeline = clone(lgbm_pipeline)
lgbm_pipeline.set_params(**lgbm_study.best_params)
lgbm_pipeline.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000178 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 595
[LightGBM] [Info] Number of data points in the train set: 937, number of used features: 24
[LightGBM] [Info] Start training from score 281999.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=(('int', RobustScaler(),
                                                  ['fuel_consumption_km_l',
                                                   'horsepower', 'displacement',
                                                   'gears', 'torque',
                                                   'passengers', 'doors',
                                                   'wheels', 'km', 'age']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['touch_screen',
                                                   'navigation_system',
                                                   'push_start',
                                                   'rear_cupholders', 'sunroof',
                                                   'rear_sensor', 'start_stop',
                                                   'turbo', 'seat_material...
                                                   'electric_parking_brake',
                                                   'electric_locks'])),
                                   verbose_feature_names_out=False)),
                ('regressor',
                 LGBMRegressor(alpha=0.5, colsample_bytree=0.5848404052757482,
                               learning_rate=0.08295730914724425, max_depth=8,
                               min_split_gain=0.00012883060884643252,
                               n_estimators=989, objective='quantile',
                               reg_alpha=8.442355086163822e-05,
                               reg_lambda=0.21318302871006012,
                               subsample=0.1949654146226087))])

In [ ]:
preds_val = lgbm_pipeline.predict(X_val)
mlu.get_metrics_pd(y_val, preds_val, 'LGBM Pipeline Optuna Metrics')

,LGBM Pipeline Optuna Metrics
MSE,1.782498e+09
R^2,8.740000e-01
MAE,2.665953e+04
RMSE,4.221964e+04


In [ ]:
# save model
LGBM_OPTUNA_DIR = NEW_MODELS_DIR.getsyspath('3_lgbm_optuna_bcu.joblib')
joblib.dump(cat_int_pipeline, LGBM_OPTUNA_DIR)